In [28]:
!pip install transformers

In [29]:
!pip install torch

In [30]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [31]:
import os
import pdfplumber
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import re
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from wordcloud import WordCloud
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [32]:
def extract_text_from_pdfs( folder_path ):
    """
    指定フォルダ内の複数PDFからテキストを抽出する。
    """
    pdf_texts = {}
    for file_name in os.listdir( folder_path ):
        if file_name.endswith( '.pdf' ):  # PDFファイルのみ対象
            with pdfplumber.open(os.path.join( folder_path, file_name) ) as pdf:
                text = ''.join([ page.extract_text () for page in pdf.pages if page.extract_text()])
                if text.strip():  # テキストが空でない場合のみ
                    pdf_texts[file_name] = text
    return pdf_texts

In [33]:
folder_path = r"C:\Users\sk062\OneDrive\デスクトップ\資料\Note info"  # 論文PDFを保存しているフォルダ

In [ ]:
pdf_texts = extract_text_from_pdfs(folder_path)

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def extract_tfidf_features_with_custom_stopwords(texts, additional_stopwords=None):
    """
    論文テキストをTF-IDFベクトルに変換して特徴量を抽出する。
    追加のストップワードを指定することが可能。

    パラメータ:
    texts : list of str
        処理対象のテキストデータのリスト。
    additional_stopwords : list of str, optional
        追加で除去したいストップワードのリスト（デフォルトはNone）。

    戻り値:
    tfidf_matrix : scipy.sparse matrix
        TF-IDF ベクトル化されたテキストデータ。
    vectorizer : TfidfVectorizer
        TF-IDF ベクトライザー（後から単語リストを取得可能）。
    """
    # ストップワードの設定
    stop_words = list(ENGLISH_STOP_WORDS)
    if additional_stopwords:
        stop_words.extend(additional_stopwords)  # リストに追加のストップワードを加える

    # TF-IDFベクトライザーの定義
    vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=5000)
    tfidf_matrix = vectorizer.fit_transform(texts)

    return tfidf_matrix, vectorizer


In [ ]:
texts = list(pdf_texts.values())  # 論文の本文
filenames = list(pdf_texts.keys())  # 対応するファイル名

In [ ]:
# 1. 各論文の文字数をカウント
char_counts = {filename: len(text) for filename, text in pdf_texts.items()}

# 2. TF-IDF特徴量抽出（カスタムストップワードなし）
tfidf_matrix, vectorizer = extract_tfidf_features_with_custom_stopwords(pdf_texts)

# 各論文の上位10単語を取得
feature_names = np.array(vectorizer.get_feature_names_out())  # 単語リスト
top_words_per_doc = {}

for i, filename in enumerate(filenames):
    # TF-IDFスコアが高い単語のインデックスを取得
    sorted_indices = np.argsort(tfidf_matrix[i].toarray()).flatten()[::-1]
    top_words = feature_names[sorted_indices[:20]]  # 上位10単語を取得
    top_words_per_doc[filename] = top_words

# 結果を表示
for filename in filenames:
    print(f"論文: {filename}")
    print(f"文字数: {char_counts[filename]}")
    print(f"最頻ワードTOP10: {', '.join(top_words_per_doc[filename])}")
    print("-" * 50)


In [ ]:
def count_citations(text):
    """
    論文の引用数をカウントする（IEEEスタイルやAPAスタイルに対応）
    """
    # IEEEスタイルの引用（例: [1], [2,3]）
    ieee_citations = re.findall(r'\[\d+(?:,\s*\d+)*\]', folder_path)

    # APAスタイルの引用（例: (Smith, 2020)）
    apa_citations = re.findall(r'\([A-Za-z]+,?\s*\d{4}\)', folder_path)

    # 自然科学系論文のパターン (例: Smith et al., 2020)
    nature_citations = re.findall(r'[A-Za-z]+ et al\.,?\s*\d{4}', folder_path)

    # 総引用数を合計
    total_citations = len(ieee_citations) + len(apa_citations) + len(nature_citations)
    return total_citations

# 各論文の引用数をカウント
citation_counts = {filename: count_citations(text) for filename, text in pdf_texts.items()}

# 結果を表示
for filename in filenames:
    print(f"論文: {filename}")
    print(f"引用数: {citation_counts[filename]}")
    print("-" * 50)


In [ ]:
# CSVファイルの読み込み
csv_file_path = r"C:\Users\sk062\OneDrive\デスクトップ\9c2e4d3d-864b-490e-b35a-b141df14fc43_Export-00b74899-a19a-4d27-9b59-9b23b5916742\Note paper (ai) 174225966a7b8067b3ecd5385709bbb4.csv"
df = pd.read_csv(csv_file_path)

In [ ]:
df

In [ ]:
# ❷ 追加するデータを整理
data = []
summarizer = pipeline('summarization', model="facebook/bart-large-cnn")  # 要約モデル